# Performance optimisation in Julia

We will cover:
- profiling
- run-time classes
- code tuning

<div class="alert alert-block alert-info" title="Aside 1">

These slides are a [Jupyter notebook](https://jupyter.org/); a browser-based computational notebook.

Code cells are executed by putting the cursor into the cell and hitting `shift + enter`.  For more
info see the [documentation](https://jupyter-notebook.readthedocs.io/en/stable/).

</div>

## Profiling

When you want to improve the performance of a program, the first important thing to do is to find out which part of your program is taking the most time to execute. This can be done by profiling. Profiling is also important when comparing the performance of two different implementations of the same task.

Very simple profiling can be done using the `@time` macro, which simply prints out how long a given statement took to execute. More advanced analysis is possible with `@profile`, which gives an insight into which part of the code required how much processing time.

### Exercise 1: converting to Celsius

Imagine you are working with a data source that uses Fahrenheit, but your calculations require degrees Celsius. How do you best convert a large number of values from one to the other?

In [1]:
using Random, Profile

## Option A: the "naive" function using a for loop and building up the result vector as we go
function converttocelsiusA(fahrenheit::Vector)
    celsius = []
    for f in fahrenheit
        push!(celsius, (f-32)*(5/9))
    end
    celsius
end

## Option B: using an anonymous function and `map`
function converttocelsiusB(fahrenheit::Vector)
    celsius = f -> (f-32)*(5/9)
    map(celsius, fahrenheit)
end

## create a random vector of 1 million numbers
fahrenheits = rand(0:250, 1000000)

## precompile the functions to save time when you run
## them for the first time
precompile(converttocelsiusA, (Vector,));
precompile(converttocelsiusB, (Vector,));

In [4]:
## Note: adding a semi-colon after a line of code suppresses
## the REPL output - in this case, we don't want to see the array
## that is produced, we're just interested in the runtime

## then use the @time macro to find out how efficient each function is
@time(converttocelsiusA(fahrenheits));
@time(converttocelsiusB(fahrenheits));

  0.023582 seconds (1.00 M allocations: 25.040 MiB)
  0.005190 seconds (2 allocations: 7.629 MiB, 69.98% gc time)


Run the code above several times to get a feel for how the execution times change, as there is always some random fluctuation. (Also, the first run is always slower because it includes the compilation time.) Still, you should see that the second option is a lot faster.

However, vectorised code can be harder to write and understand than code that uses loops. So is it possible to speed up the first function? Let's take a look at this adapted version:

In [6]:
## Option C / A2: uses a loop like the first function, but preallocates the vector to avoid using push!()
function converttocelsiusA2(fahrenheit::Vector)
    celsius = Vector{Float64}(undef, length(fahrenheit))
    for f in eachindex(fahrenheit)
        celsius[f] = (fahrenheit[f]-32)*(5/9)
    end
    celsius
end

converttocelsiusA2(fahrenheits);

@time(converttocelsiusA2(fahrenheits));

  0.003631 seconds (2 allocations: 7.629 MiB)


Avoiding the allocations has allowed us to speed up the loop to a similar level as the vectorised code.

### Exercise 2: Plant reproduction

The `@profile` macro works by taking snapshots of the execution of your code every few milliseconds. At every snapshot, it records which function is currently executing, and which function(s) called this function. When your code is done, you thus get an insight into how much time the computer spends executing each subfunction.

Consider this mini-model of a plant population, in which only the largest plants can reproduce. Which part of the model is more computationally expensive - finding the largest plants or letting them reproduce? Which parameters influence this?

*(As before, execute this code several times to allow for compilation.)*

In [10]:
## A simple plant type
struct Plant
    id::Int
    size::Int
    seeds::Int
end

## Initialise 10 million plants
biome = [Plant(p, rand(1:10000), 1000) for p in 1:10000000]

## Return the indices of the largest plants
function findlargest(plants::Vector{Plant})
    maxsize = 0
    ids = []
    for p in eachindex(plants)
        if plants[p].size > maxsize
            maxsize = plants[p].size
            ids = [p]
        elseif plants[p].size == maxsize
            push!(ids, p)
        end
    end
    return ids
end

## Return a vector filled with one new plant for every seed produced by the input plants
function reproduce!(plants::Vector{Plant})
    offspring = []
    for p in plants
        for s in 1:p.seeds
            push!(offspring, Plant(length(plants)+length(offspring), p.size, p.seeds))
        end
    end
    offspring
end

## Find the largest plants and let only these reproduce
function updateplants(plants::Vector{Plant})
    largest = findlargest(plants)
    reproduce!(plants[largest])
end

## Finally, profile our mini-model and show the results
Profile.clear()
@profile updateplants(biome);
Profile.print(format=:flat)

 Count  Overhead File                    Line Function
 =====  ======== ====                    ==== ========
    94         0 @Base/Base.jl            495 include(mod::Module, _path::Strin…
     6         6 @Base/array.jl          1026 __inbounds_setindex!
     6         6 @Base/array.jl          1072 _growend!
     1         1 @Base/array.jl             ? push!
     6         0 @Base/array.jl          1126 push!
     6         0 @Base/array.jl          1127 push!
    94        56 @Base/boot.jl            385 eval
    94         0 @Base/client.jl          552 _start()
    94         0 @Base/client.jl          318 exec_options(opts::Base.JLOptions)
     2         0 …ractinterpretation.jl  2162 abstract_call(interp::Core.Compil…
     2         0 …ractinterpretation.jl  2169 abstract_call(interp::Core.Compil…
     2         0 …ractinterpretation.jl  2354 abstract_call(interp::Core.Compil…
     2         0 …ractinterpretation.jl    95 abstract_call_gf_by_type(interp::…
     2         0 …r

# Optimising code

### Exercise 3: Largest prime factor

**The prime factors of 13195 are 5, 7, 13, and 29. What is the largest prime factor of the number 600851475143?**

*(Taken from: https://projecteuler.net/problem=3)*

Let's try a first implementation to solve this problem:

In [29]:
isfactor(x, y) = iszero(x % y)

function isprime(n)
    n == 1 && return false
    for i in 2:(n-1)
        isfactor(n, i) && return false
    end
    return true
end

function factors(x)
    factorlist = []
    for i in 1:x
        isfactor(x, i) && push!(factorlist, i)
    end
    factorlist
end

function largestprimefactor(n)
    primefactors = []
    for f in factors(n)
        isprime(f) && push!(primefactors, f)
    end
    return maximum(primefactors)
end

largestprimefactor (generic function with 1 method)

In [30]:
@time largestprimefactor(13195)

  0.008299 seconds (2.62 k allocations: 184.500 KiB, 98.90% compilation time)


29

This answer is correct. However, trying to apply our function `largestprimefactor()` to 600851475143 runs and runs and runs and runs... We need something faster. Look at the functions again. What do you see that may be computationally expensive? Can you try writing your own version of these functions that is faster?

In [22]:
isfactor(x, y) = iszero(x % y)

#TODO write your own functions to solve the problem.

function largestprimefactor2(n)
    #TODO complete this function
end

largestprimefactor2 (generic function with 1 method)

In [23]:
# Test your implementation using this code cell. Is your version faster than the first?
@time largestprimefactor2(13195)

  0.000001 seconds


Once you have tried optimising this yourself, here is my best version. How did I optimise the code?

In [35]:
function isprime3(n::Int64)
    (n == 1 || iszero(n % 2)) && return false
    i = 3
    while i < n
        iszero(n % i) && return false
        i += 2
    end
    return true
end

function largestprimefactor3(n::Int64)
    for j in 1:n
        (iszero(n % j) && isprime3(div(n,j))) && return div(n,j)
    end
    return nothing
end


largestprimefactor3 (generic function with 2 methods)

In [37]:
@time largestprimefactor3(13195)
@time largestprimefactor3(600851475143)

  0.000003 seconds
  0.679900 seconds


6857

### Tuning tips

#### Common miscreants

- input/output operations (especially to disk)
- nested loops
- array allocation

#### Strategies

- buffer output before writing
- structure data to reduce the runtime class
- reduce array allocations
- cache calculations so you only need to do them once
- parallelise your code
- if memory is an issue: compress data